In [2]:
import numpy as np
import pandas as pd

# score 함수 정의

In [3]:
# get rmsle
def rmsle(y_true, y_pred):
    return np.sqrt(np.mean(np.power(np.log1p(y_true) - np.log1p(y_pred), 2)))

In [4]:
def get_ECLO(x1, x2, x3, x4):
    return x1*10+x2*5+x3*3+x4

# 전처리

In [5]:
train_df = pd.read_csv('../EDA/dataset/train.csv', index_col="ID")
test_df = pd.read_csv('../EDA/dataset/test.csv', index_col="ID")

In [6]:
label1 = train_df['사망자수']
label2 = train_df['중상자수']
label3 = train_df['경상자수']
label4 = train_df['부상자수']

In [7]:
train_df.drop(["사망자수", "중상자수", "경상자수", "부상자수", "ECLO"], axis=1, inplace=True)

In [8]:
exclude_columns = train_df.columns.drop(["사고일시", "요일", "기상상태", "도로형태", "노면상태", "사고유형"])
train_df.drop(exclude_columns, axis=1, inplace=True)

In [9]:
def date_split(dataset):
    dataset["month"] = dataset["사고일시"].apply(lambda x: int(x.split("-")[1]))
    dataset["day"] = dataset["사고일시"].apply(lambda x: int(x.split(" ")[1]))
    return dataset
train_df = date_split(train_df)
test_df = date_split(test_df)
train_df.drop(["사고일시"], axis=1, inplace=True)
test_df.drop(["사고일시"], axis=1, inplace=True)

In [10]:
test_df.drop("시군구", axis=1, inplace=True)
test_df.head()

,요일,기상상태,도로형태,노면상태,사고유형,month,day
ID,,,,,,,
ACCIDENT_39609,토요일,맑음,교차로 - 교차로안,건조,차대사람,1,1
ACCIDENT_39610,토요일,맑음,단일로 - 기타,건조,차대사람,1,1
ACCIDENT_39611,토요일,맑음,교차로 - 교차로안,건조,차대차,1,4
ACCIDENT_39612,토요일,맑음,단일로 - 기타,건조,차대차,1,4
ACCIDENT_39613,토요일,맑음,교차로 - 교차로안,건조,차대차,1,6


In [11]:
train_df.head()

,요일,기상상태,도로형태,노면상태,사고유형,month,day
ID,,,,,,,
ACCIDENT_00000,화요일,맑음,단일로 - 기타,건조,차대사람,1,0
ACCIDENT_00001,화요일,흐림,단일로 - 기타,건조,차대사람,1,0
ACCIDENT_00002,화요일,맑음,단일로 - 기타,건조,차대사람,1,1
ACCIDENT_00003,화요일,맑음,단일로 - 기타,건조,차대차,1,2
ACCIDENT_00004,화요일,맑음,단일로 - 기타,건조,차대차,1,4


# model

In [12]:
# 카테고리 변수 트리 모델용으로 변환
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
train_df = ohe.fit_transform(train_df)
test_df = ohe.transform(test_df)
train_df

<39609x69 sparse matrix of type '<class 'numpy.float64'>'
	with 277263 stored elements in Compressed Sparse Row format>

In [13]:
# model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

rf1 = RandomForestRegressor(n_estimators= 350, random_state=42, n_jobs=-1)
rf2 = RandomForestRegressor(n_estimators= 350, random_state=42, n_jobs=-1)
rf3 = RandomForestRegressor(n_estimators= 350, random_state=42, n_jobs=-1)
rf4 = RandomForestRegressor(n_estimators= 350, random_state=42, n_jobs=-1)
train_X, val_X, train_y, val_y1 = train_test_split(train_df, label1, test_size=0.2, random_state=42)
rf1.fit(train_X, train_y)
train_X, val_X, train_y, val_y2 = train_test_split(train_df, label2, test_size=0.2, random_state=42)
rf2.fit(train_X, train_y)
train_X, val_X, train_y, val_y3 = train_test_split(train_df, label3, test_size=0.2, random_state=42)
rf3.fit(train_X, train_y)
train_X, val_X, train_y, val_y4 = train_test_split(train_df, label4, test_size=0.2, random_state=42)
rf4.fit(train_X, train_y)
pred1 = rf1.predict(val_X)
pred2 = rf2.predict(val_X)
pred3 = rf3.predict(val_X)
pred4 = rf4.predict(val_X)
pred = get_ECLO(pred1, pred2, pred3, pred4)
val_y = get_ECLO(val_y1, val_y2, val_y3, val_y4)


In [14]:
# w * pred 최소값 찾기
from scipy.optimize import minimize
def objective(params):
    return rmsle(val_y, params[0]*pred + params[1])
initial = [1, 0]
sol = minimize(objective, initial)
print(sol.x, sol.fun)
# 0.45136445

[0.11047476 3.58626527] 0.45136445295250854


In [15]:
# submission = model.predict(test_df)
# test_df = pd.read_csv('../EDA/dataset/test.csv', index_col="ID")
# submission = pd.DataFrame(submission, index=test_df.index, columns=["ECLO"])
# submission.to_csv("submission.csv", index=True)

public: 0.4464058375

private: 0.447986566